# Extracción de caracteristicas del texto

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

ROOT_PATH = '/content/drive/My Drive/Meli-Challenge/'

In [0]:
# data = pd.read_csv('train.csv')
data = pd.read_csv(ROOT_PATH + 'train2.csv')

In [0]:
data.head()

,Unnamed: 0,title,label_quality,language,category,length
0,0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS,60
1,1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS,33
2,2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES,36
3,3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS,54
4,4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES,58


In [0]:
data[data['length'] > 80].iloc[25]['title']

'Cabina Secadora Uñas Uv Led 72 W Timer Display Esculpidas Gelificadas Acrílicas Semipermanente'

## Text Cleaning

* Dejar todo en minuscula
* Quitar tildes
* Quitar caracteres de puntuacion

In [0]:
data['title'] = data['title'].str.lower()
l = ['á', 'é', 'í', 'ó', 'ú']
l2 = ['a', 'e', 'i', 'o', 'u']
for i in range(5):
  data['title'] = data['title'].str.replace(l[i], l2[i])
  
signos = list("?:!.,;")
for i in signos:
  data['title'] = data['title'].str.replace(i, '')

## Lemmatization

In [0]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
lemmatizer = WordNetLemmatizer()

In [0]:
nrows = data.shape[0]
lemmatized_text_list = []

for row in range(nrows):
  lemmatized_list = []
  
  # 3 pasos, primero separar el titulo en palabras
  text = data.loc[row]['title']
  text_words = text.split(' ')
  
  # siguiente paso es lematizar cada palabra
  for word in text_words:
    lemmatized_list.append(lemmatizer.lemmatize(word, pos="v"))
    
  # ultimo: unir en una sola frase las palabras lematizadas
  lemmatized_text = " ".join(lemmatized_list)
  
  lemmatized_text_list.append(lemmatized_text)

In [0]:
data['lemmatized_title'] = lemmatized_text_list

In [0]:
data.pop('Unnamed: 0');

In [0]:
data.head()

,title,label_quality,language,category,length,lemmatized_title
0,hidrolavadora lavor one 120 bar 1700w bomba a...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS,60,hidrolavadora lavor one 120 bar 1700w bomba a...
1,placa de sonido - behringer umc22,unreliable,spanish,SOUND_CARDS,33,placa de sonido - behringer umc22
2,maquina de lavar electrolux 12 kilos,unreliable,portuguese,WASHING_MACHINES,36,maquina de lavar electrolux 12 kilos
3,par disco de freio diant vent gol 8v 08/ frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS,54,par disco de freio diant vent gol 8v 08/ frema...
4,flashes led pestañas luminoso falso pestañas p...,unreliable,spanish,FALSE_EYELASHES,58,flash lead pestañas luminoso falso pestañas pa...


Despues de mas de una hora se lematizo esa monda!

In [0]:
data.to_csv(ROOT_PATH + 'train3_lemmatized.csv')

_____

# Stop Words

In [0]:
lemmatized_data = pd.read_csv(ROOT_PATH + 'train3_lemmatized.csv', sep=',')

In [0]:
lemmatized_data.pop('Unnamed: 0');

In [0]:
lemmatized_data.columns

Index(['title', 'label_quality', 'language', 'category', 'length',
       'lemmatized_title'],
      dtype='object')

In [0]:
spanish_data = lemmatized_data.loc[lemmatized_data[lemmatized_data['language'] == 'spanish'].index]
portuguese_data = lemmatized_data.loc[lemmatized_data[lemmatized_data['language'] == 'portuguese'].index]

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Stop words of Spanish**

In [0]:
stop_words = list(stopwords.words('spanish'))

In [0]:
spanish_data['parsed_title'] = spanish_data['lemmatized_title']

for stop_word in stop_words:
  
  regex_exp = r"\b" + stop_word + r"\b"
  spanish_data['parsed_title'] = spanish_data['parsed_title'].str.replace(regex_exp, '')

**Stop words of Portuguese!**

Rebooting the environment

In [0]:
stop_words_portugues = list(stopwords.words('portuguese'))

In [0]:
portuguese_data['parsed_title'] = portuguese_data['lemmatized_title']

for stop_word in stop_words_portugues:
  
  regex_exp = r"\b" + stop_word + r"\b"
  portuguese_data['parsed_title'] = portuguese_data['parsed_title'].str.replace(regex_exp, '')

In [0]:
spanish_data.to_csv(ROOT_PATH + 'spanish_data.csv')

In [0]:
portuguese_data.to_csv(ROOT_PATH + 'portuguese_data.csv')

## Label Encoding

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
label_encoder = LabelEncoder()

In [0]:
spanish_data = pd.read_csv(ROOT_PATH + 'spanish_data.csv')

In [0]:
portuguese_data = pd.read_csv(ROOT_PATH + 'portuguese_data.csv')

In [0]:
spanish_data.head(2)

,Unnamed: 0,title,label_quality,language,category,length,lemmatized_title,parsed_title
0,0,hidrolavadora lavor one 120 bar 1700w bomba a...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS,60,hidrolavadora lavor one 120 bar 1700w bomba a...,hidrolavadora lavor one 120 bar 1700w bomba a...
1,1,placa de sonido - behringer umc22,unreliable,spanish,SOUND_CARDS,33,placa de sonido - behringer umc22,placa sonido - behringer umc22


In [0]:
spanish_data.drop(['Unnamed: 0', 'lemmatized_title', 'length'], axis=1, inplace=True);

In [0]:
portuguese_data.drop(['Unnamed: 0', 'lemmatized_title', 'length'], axis=1, inplace=True);

In [0]:
spanish_data.columns

Index(['title', 'label_quality', 'language', 'category', 'parsed_title'], dtype='object')

In [0]:
new_data = pd.concat([spanish_data, portuguese_data])

In [0]:
new_data.columns

Index(['title', 'label_quality', 'language', 'category', 'parsed_title'], dtype='object')

In [0]:
new_data['encoded_category'] = label_encoder.fit_transform(new_data['category'])

In [0]:
new_data.head(2)

,title,label_quality,language,category,parsed_title,encoded_category
0,hidrolavadora lavor one 120 bar 1700w bomba a...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS,hidrolavadora lavor one 120 bar 1700w bomba a...,544
1,placa de sonido - behringer umc22,unreliable,spanish,SOUND_CARDS,placa sonido - behringer umc22,1316


In [0]:
new_data['encoded_category'].value_counts().shape

(1588,)

In [0]:
new_data.to_csv(ROOT_PATH + 'Final_Data.csv')

## Spliting the data

In [0]:
new_data = pd.read_csv(ROOT_PATH + 'Final_Data.csv')

In [0]:
new_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(new_data['parsed_title'], 
                                                    new_data['encoded_category'], 
                                                    test_size=0.05, 
                                                    random_state=8)

In [0]:
# del(new_data)

## Text Representation

In [0]:
# Parameter election
ngram_range = (1,2)
min_df = 5
max_df = 0
max_features = 300

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

In [0]:
index_null = X_train[X_train.isnull()].index
X_train.drop(index_null, axis=0, inplace=True)
y_train.drop(index_null, axis=0, inplace=True)

In [0]:
index_null_test = X_test[X_test.isnull()].index
X_test.drop(index_null_test, axis=0, inplace=True)
y_test.drop(index_null_test, axis=0, inplace=True)

In [0]:
# del(index_null)

In [0]:
# del(index_null_test)

In [0]:
features_train = tfidf.fit_transform(X_train).toarray()
print(features_train.shape)

In [0]:
features_test = tfidf.transform(X_test).toarray()
print(features_test.shape)

## Another way

no se pueden usar los 20M de datos, por limitaciones de ram

In [0]:
new_data = pd.read_csv(ROOT_PATH + 'Final_Data.csv')

In [0]:
new_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [0]:
reliable_data = new_data.loc[new_data['label_quality'] == 'reliable']

In [0]:
all_cats = set(new_data['encoded_category'].value_counts().index)
reli_cats = set(reliable_data['encoded_category'].value_counts().index)

In [0]:
len(all_cats) - len(reli_cats)

193

In [0]:
extra_cats = all_cats - reli_cats
len(extra_cats)

193

In [0]:
for cat in extra_cats:
  df_slice = new_data.loc[new_data['encoded_category'] == cat]
  reliable_data = pd.concat([reliable_data, df_slice])

In [0]:
reliable_data.shape

(2386304, 6)

In [0]:
reliable_data.to_csv(ROOT_PATH + 'full_reliable_data.csv')

## New Data import

In [0]:
reliable_data = pd.read_csv(ROOT_PATH + 'full_reliable_data.csv')

In [0]:
reliable_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
reliable_data.shape

(2386304, 6)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(reliable_data['parsed_title'], 
                                                    reliable_data['encoded_category'], 
                                                    test_size=0.15, 
                                                    random_state=8)

## Text Representation AGAIN

In [0]:
# Parameter election
ngram_range = (1,2)
min_df = 0
max_df = 5
max_features = 20

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

Eliminamos los valores null

In [0]:
index_null = X_train[X_train.isnull()].index
X_train.drop(index_null, axis=0, inplace=True)
y_train.drop(index_null, axis=0, inplace=True)

In [0]:
index_null_test = X_test[X_test.isnull()].index
X_test.drop(index_null_test, axis=0, inplace=True)
y_test.drop(index_null_test, axis=0, inplace=True)

Hacemos los fit transform y transform necesarios para extraer las caracteristicas

In [19]:
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

(2028357, 20)


In [20]:
features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(357946, 20)


In [0]:
import pickle

Saving the files as Pickles

In [0]:
# X_train
with open(ROOT_PATH + 'pickles_reliable_data/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open(ROOT_PATH + 'pickles_reliable_data/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open(ROOT_PATH + 'pickles_reliable_data/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open(ROOT_PATH + 'pickles_reliable_data/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# features_train
with open(ROOT_PATH + 'pickles_reliable_data/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open(ROOT_PATH + 'pickles_reliable_data/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open(ROOT_PATH + 'pickles_reliable_data/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open(ROOT_PATH + 'pickles_reliable_data/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open(ROOT_PATH + 'pickles_reliable_data/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)